<a href="https://colab.research.google.com/github/cham2601/Power-quality-classification/blob/main/power_quality_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing data

In [140]:
import numpy as np
import pandas as pd

# load dataset
X_train = pd.read_csv("Voltage_L1_DataSet1.csv",header=None)
Y_train = pd.read_csv("OutputFor_DataSet1.csv",header=None)
X_test = pd.read_csv("Voltage_L1_DataSet2.csv",header=None)
Y_test = pd.read_csv("OutputFor_DataSet2.csv",header=None)

# X_train = pd.read_csv("Voltage_L1_train.csv",header=None)
# Y_train = pd.read_csv("output_train.csv",header=None)
# X_test = pd.read_csv("Voltage_L1_test.csv",header=None)
# Y_test = pd.read_csv("output_test.csv",header=None)
X_train = X_train.dropna()
X_test = X_test.dropna()

In [141]:
print(X_train.shape,X_test.shape,Y_train.shape,Y_test.shape)

(6000, 128) (6000, 128) (6000, 1) (6000, 1)


## Downloading libraries

In [142]:
from keras import Sequential
from keras.models import Model
from keras.layers import Dense,Input,Dropout,LSTM
from keras.utils import to_categorical
from keras.optimizers import Adam
# from tensorflow.keras.layers.experimental import RandomFourierFeatures,preprocessing
import tensorflow as tf
# import keras
import tensorflow.keras.backend as K
np.random.seed(0)

## Data preprocessing

In [143]:
x_train = np.array(X_train).reshape(X_train.shape[0],X_train.shape[1],1)
x_test = np.array(X_test).reshape(X_test.shape[0],X_test.shape[1],1)

y_train = to_categorical(Y_train)
y_test = to_categorical(Y_test)

input_shape = x_train.shape[1:]
classes = y_train.shape[1]
print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)

(6000, 128, 1) (6000, 128, 1) (6000, 6) (6000, 6)


## LSTM model

### Building

In [120]:
def LSTMmodel(input_shape,classes):
  input = Input(shape=input_shape)
  out = LSTM(units = 128,return_sequences=True)(input)
  out = Dropout(rate = 0.3)(out)
  out = LSTM(units = 128,return_sequences=False)(out)
  out = Dropout(rate = 0.3)(out)
  out = Dense(classes,'softmax')(out)

  model = Model(input,out)
  return model

## Compiling model for dataset1

In [121]:
model1 = LSTMmodel(input_shape,classes)
model1.compile(optimizer=Adam(learning_rate=0.001), loss="categorical_crossentropy", metrics=["accuracy"])
model1.summary()

Model: "model_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_24 (InputLayer)        [(None, 256, 1)]          0         
_________________________________________________________________
lstm_46 (LSTM)               (None, 256, 128)          66560     
_________________________________________________________________
dropout_46 (Dropout)         (None, 256, 128)          0         
_________________________________________________________________
lstm_47 (LSTM)               (None, 128)               131584    
_________________________________________________________________
dropout_47 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 7)                 903       
Total params: 199,047
Trainable params: 199,047
Non-trainable params: 0
____________________________________________________

In [122]:
# pip install livelossplot

### Training

In [241]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from livelossplot import PlotLossesKeras

chkpt = ModelCheckpoint("LSTM_model1weights.h5", monitor='val_loss',verbose=1, save_best_only=True, save_weights_only=True, mode='min')

early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=1, verbose=0, mode='max', baseline=None, restore_best_weights=False)

callbacks = [chkpt]

model1.fit(x_train,y_train,epochs=30,validation_split=0.3,batch_size=128)

33/33 [==============================] - 72s 2s/step - loss: 0.0664 - accuracy: 0.9776 - val_loss: 0.0440 - val_accuracy: 0.9833


### Testing

In [ ]:
loss1,acc1 = model1.evaluate(x_test,y_test,batch_size=32)

### Saving weights and model

In [104]:
model1.save('LSTM.h5')
# model_json = model1.to_json()
# with open("model.json", "w") as json_file:
#     json_file.write(model_json)

### loading weights and evaluating

In [102]:
model1.load_weights('LSTM_weights1.h5')
model1.evaluate(x_test,y_test)

188/188 [==============================] - 50s 261ms/step - loss: 0.0419 - accuracy: 0.9871


[0.04287343472242355, 0.9856666922569275]

In [103]:
model1.load_weights('LSTM.h5')
model1.evaluate(x_test,y_test)

188/188 [==============================] - 49s 259ms/step - loss: 0.0429 - accuracy: 0.9857


[0.04287343472242355, 0.9856666922569275]

## Dataset2

In [123]:
def LSTM_SVMmodel(input_shape,classes):
  input = Input(shape=input_shape)
  out = LSTM(units = 128,return_sequences=True)(input)
  out = Dropout(rate = 0.3)(out)
  out = LSTM(units = 128,return_sequences=False)(out)
  out = Dropout(rate = 0.3)(out)
  out = Dense(classes, use_bias=False, activation='linear', name='svm')(out)

  model = Model(input,out)
  return model

In [124]:
def svm_loss(layer):
    weights = layer.weights[0]
    weights_tf = tf.convert_to_tensor(weights)
    
    def categorical_hinge_loss(y_true, y_pred):
        pos = K.sum(y_true * y_pred, axis=-1)
        neg = K.max((1.0 - y_true) * y_pred, axis=-1)
        hinge_loss = K.mean(K.maximum(0.0, neg - pos + 1), axis=-1)
        regularization_loss = 0.5*(tf.reduce_sum(tf.square(weights_tf)))
        return regularization_loss + 0.4*hinge_loss
    
    return categorical_hinge_loss

In [164]:
model2 = LSTM_SVMmodel(input_shape,classes)

metrics = ['accuracy']
# optimizer = tf.keras.optimizers.RMSprop(lr=2e-3, decay=1e-5)
optimizer = Adam(learning_rate=0.001)

model2.compile(optimizer=optimizer, loss=svm_loss(model.get_layer('svm')), metrics=metrics)
model2.summary()

Model: "model_34"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_35 (InputLayer)        [(None, 128, 1)]          0         
_________________________________________________________________
lstm_60 (LSTM)               (None, 128, 128)          66560     
_________________________________________________________________
dropout_68 (Dropout)         (None, 128, 128)          0         
_________________________________________________________________
lstm_61 (LSTM)               (None, 128)               131584    
_________________________________________________________________
dropout_69 (Dropout)         (None, 128)               0         
_________________________________________________________________
svm (Dense)                  (None, 6)                 768       
Total params: 198,912
Trainable params: 198,912
Non-trainable params: 0
____________________________________________________

In [129]:
# from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
# from livelossplot import PlotLossesKeras

# chkpt = ModelCheckpoint("LSTM_svmweights1.h5", monitor='val_loss',verbose=1, save_best_only=True, save_weights_only=True, mode='min')
# callbacks = [PlotLossesKeras(),chkpt]

history = model2.fit(x_train, y_train,
                    batch_size=64,
                    epochs=2,
                    validation_split=0.3
                    )

Epoch 1/2
66/66 [==============================] - 68s 1s/step - loss: 5.2535 - accuracy: 0.4593 - val_loss: 5.5371 - val_accuracy: 0.0156
Epoch 2/2
66/66 [==============================] - 68s 1s/step - loss: 5.2299 - accuracy: 0.4990 - val_loss: 5.4988 - val_accuracy: 0.0061


In [165]:
from keras.models import load_model
model2.save('LSTM_SVM.h5')
# del model2
# model2 = load_model('LSTM_SVM.h5')
# model2.evaluate(x_test,y_test)

## SVM

In [158]:
def model_1(x_input):
    x = Dense(512, activation='relu')(x_input)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    x_out = Dense(256, activation='relu')(x)
    x_out = Dropout(0.5)(x)
    return x_out

In [19]:
def model(input_shape,classes):
  input = Input(shape=input_shape)
  # out = normalizer(input)
  out = LSTM(units = 256,return_sequences=True)(input)
  out = Dropout(rate = 0.3)(out)
  out = LSTM(units = 256,return_sequences=False)(out)
  out = Dropout(rate = 0.3)(out)
  out = Dense(classes, use_bias=False, activation='linear', name='svm')(out)

  model = Model(input,out)
  return model

In [20]:
model = model(input_shape,classes)

In [21]:
def svm_loss(layer):
    weights = layer.weights[0]
    weights_tf = tf.convert_to_tensor(weights)
    
    def categorical_hinge_loss(y_true, y_pred):
        pos = K.sum(y_true * y_pred, axis=-1)
        neg = K.max((1.0 - y_true) * y_pred, axis=-1)
        hinge_loss = K.mean(K.maximum(0.0, neg - pos + 1), axis=-1)
        regularization_loss = 0.5*(tf.reduce_sum(tf.square(weights_tf)))
        return regularization_loss + 0.4*hinge_loss
    
    return categorical_hinge_loss

In [24]:
metrics = ['accuracy']
optimizer = tf.keras.optimizers.RMSprop(lr=2e-3, decay=1e-5)
#optimizer = tf.train.AdamOptimizer(1.e-3)

model.compile(optimizer=optimizer, loss=svm_loss(model.get_layer('svm')), metrics=metrics)
model.summary()

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 128, 1)]          0         
_________________________________________________________________
lstm_16 (LSTM)               (None, 128, 256)          264192    
_________________________________________________________________
dropout_16 (Dropout)         (None, 128, 256)          0         
_________________________________________________________________
lstm_17 (LSTM)               (None, 256)               525312    
_________________________________________________________________
dropout_17 (Dropout)         (None, 256)               0         
_________________________________________________________________
svm (Dense)                  (None, 6)                 1536      
Total params: 791,040
Trainable params: 791,040
Non-trainable params: 0
_____________________________________________________

In [25]:
batch_size = 64
epochs = 4

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.3)
# score = model.evaluate(x_test, y_test, verbose=0)
# print('Test loss:', score[0])
# print('Test accuracy:', score[1])

Epoch 1/4
66/66 [==============================] - 91s 1s/step - loss: 6.9872 - accuracy: 0.3326 - val_loss: 6.7612 - val_accuracy: 0.5700
Epoch 2/4
66/66 [==============================] - 87s 1s/step - loss: 6.7284 - accuracy: 0.6359 - val_loss: 6.6415 - val_accuracy: 0.7006
Epoch 3/4
66/66 [==============================] - 87s 1s/step - loss: 6.6723 - accuracy: 0.6770 - val_loss: 6.6242 - val_accuracy: 0.6661
Epoch 4/4
66/66 [==============================] - 87s 1s/step - loss: 6.6058 - accuracy: 0.7628 - val_loss: 6.5536 - val_accuracy: 0.8517


## CNN

In [153]:
def CNNmodel():
  input = tf.keras.layers.Input(shape=input_shape)
  out = tf.keras.layers.Conv1D(
      filters=32, kernel_size=7, padding="same", strides=2, activation="relu"
  )(input)
  out = tf.keras.layers.Dropout(rate=0.2)(out)
  out = tf.keras.layers.Conv1D(
      filters=16, kernel_size=7, padding="same", strides=2, activation="relu"
  )(out)
  out = tf.keras.layers.Conv1DTranspose(
      filters=16, kernel_size=7, padding="same", strides=2, activation="relu"
  )(out)
  out = tf.keras.layers.Dropout(rate=0.2)(out)
  out = tf.keras.layers.Conv1DTranspose(
      filters=32, kernel_size=7, padding="same", strides=2, activation="relu"
  )(out)
  out = tf.keras.layers.Conv1DTranspose(filters=1, kernel_size=7, padding="same")(out)
  out = LSTM(units=128)(out)
  out = Dense(classes,activation='softmax')(out)
  model = Model(input,out)
  return model

In [161]:
CNNmodel1 = CNNmodel()
# CNNmodel1.compile(optimizer=tf.keras.optimizers.RMSprop(),
#               loss=tf.keras.losses.SparseCategoricalCrossentropy(),
#               metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
CNNmodel1.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss="mse", metrics=['accuracy'])
CNNmodel1.summary()

Model: "model_33"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_34 (InputLayer)        [(None, 128, 1)]          0         
_________________________________________________________________
conv1d_14 (Conv1D)           (None, 64, 32)            256       
_________________________________________________________________
dropout_66 (Dropout)         (None, 64, 32)            0         
_________________________________________________________________
conv1d_15 (Conv1D)           (None, 32, 16)            3600      
_________________________________________________________________
conv1d_transpose_17 (Conv1DT (None, 64, 16)            1808      
_________________________________________________________________
dropout_67 (Dropout)         (None, 64, 16)            0         
_________________________________________________________________
conv1d_transpose_18 (Conv1DT (None, 128, 32)           361

In [155]:
# print(X_train.shape,Y_train.shape)
y_train.shape

(6000, 6)

In [162]:
# CNNchkpt = ModelCheckpoint("CNN_model1weights.h5", monitor='val_loss',verbose=1, save_best_only=True, save_weights_only=True, mode='min')
history = CNNmodel1.fit(x_train,y_train,epochs=10,batch_size=64)

Epoch 1/10
94/94 [==============================] - 21s 205ms/step - loss: 0.1308 - accuracy: 0.2602
Epoch 2/10
94/94 [==============================] - 19s 204ms/step - loss: 0.1208 - accuracy: 0.3275
Epoch 3/10
94/94 [==============================] - 19s 204ms/step - loss: 0.1034 - accuracy: 0.4107
Epoch 4/10
94/94 [==============================] - 19s 205ms/step - loss: 0.1035 - accuracy: 0.4225
Epoch 5/10
94/94 [==============================] - 19s 204ms/step - loss: 0.1026 - accuracy: 0.4282
Epoch 6/10
94/94 [==============================] - 19s 204ms/step - loss: 0.1030 - accuracy: 0.4203
Epoch 7/10
94/94 [==============================] - 19s 205ms/step - loss: 0.1031 - accuracy: 0.4179
Epoch 8/10
94/94 [==============================] - 19s 206ms/step - loss: 0.1028 - accuracy: 0.4326
Epoch 9/10
94/94 [==============================] - 19s 205ms/step - loss: 0.1034 - accuracy: 0.4103
Epoch 10/10
94/94 [==============================] - 19s 202ms/step - loss: 0.1032 - accura